In [29]:
import pandas as pd
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import LlamaCpp
from tqdm import tqdm

In [51]:
llm = LlamaCpp(
    model_path="../models/Mixtral-8x7B-Instruct-v0.1-GGUF/mixtral-8x7b-instruct-v0.1.Q6_K.gguf",
    n_gpu_layers=50,
    n_batch=512,
    n_ctx=4048,
    f16_kv=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
    temperature = 0
)

llama_model_loader: loaded meta data with 26 key-value pairs and 995 tensors from ../models/Mixtral-8x7B-Instruct-v0.1-GGUF/mixtral-8x7b-instruct-v0.1.Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q6_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:          blk.0.ffn_gate.0.weight q6_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    2:          blk.0.ffn_down.0.weight q6_K     [ 14336,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_up.0.weight q6_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    4:          blk.0.ffn_gate.1.weight q6_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    5:          blk.0.ffn_down.1.weight q6_K     [ 14336,  4096,     1,     1 ]
llama_model_loader: - tensor    6:            blk.0.ffn_up.1.weight q6_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    7:          blk.0.ffn_ga

In [3]:
df = pd.read_csv('./data_merge.csv')

In [4]:
df['part_2'] = [x.split('III')[0] for x in df['text']]
df['part_2'] = [x.split('II')[-1] for x in df['part_2']]

In [5]:
schema = Object(
    id="project_info",
    description="detailed information about project.",
    attributes=[
        Text(
            id="description",
            description="Detailed description of project.",
            examples=[("1. Масштаб проекта: проект расположен на экспериментальном заводе No 2 на промышленной базе компании &lt; &lt; Хинон-Юнк &gt; &gt; в Чжуншань, провинция Гуандун\n Внутри. Проект строительства многосвободной системы динамического подъема требует строгой калибровки антиядерной базы и определения структурно-конструкторских составов. Дело, в котором определены требования к строительным процессам, техническая форма, например, проектная программа и проектирование архитектурных чертежей, необходимых для этого проекта.\n2004/2005 год Этот проект закупок включает два этапа проектной работы, первый этап которой начинается после подписания контракта и второй этап - с участием участников торгов.", 
                       "Проект строительства многосвободной системы динамического подъема требует строгой калибровки антиядерной базы и определения структурно-конструкторских составов.\n Дело, в котором определены требования к строительным процессам, техническая форма, например, проектная программа и проектирование архитектурных чертежей, необходимых для этого проекта.\n2004/2005 год Этот проект закупок включает два этапа проектной работы, первый этап которой начинается после подписания контракта и второй этап - с участием участников торгов.")],
        ),
        Text(
            id="requirements",
            description="Technical and design requirements of project.",
            examples=[("Содержание и сфера охвата запроса:\n1) Реакционная базовая программа и проектирование чертежей.\n2) Техническое руководство по проектированию чертежей и фундаментальному строительству на месте.", 
                       "1) Реакционная базовая программа и проектирование чертежей.\n2) Техническое руководство по проектированию чертежей и фундаментальному строительству на месте.")],
        ),
        Text(
            id="stages",
            description="Project stages and timelines.",
            examples=[("Этот проект закупок включает два этапа проектной работы, первый этап которой начинается после подписания контракта и второй этап - с участием участников торгов.",
                       "2"),
                       ("данный тендерный проект разделен на 1 пункт", "1"), 
                       ("Данный тендерный проект делится на четыре сегмента:", '4')
                       ]
        ),
        Text(
            id="delivery_time",
            description="Product delivery time.",
            examples=[("Срок поставки: срок поставки в течение 45 рабочих дней, в зависимости от того, был ли заключен контракт с участником торгов","45"), 
                      ("Дата запланированной поставки: ожидаемая от покупателя дата поставки составляется на месяц.", "30"),
                      ("Планируемый период\n78 календарных дней работы", "78")
                      ]
        ),
        Text(
            id="start_date",
            description="Start date of work.",
            examples=[("Дата начала работ 15 мая 2023 года\nДата завершения проекта 31 июля 2023 года", "15.05.2023"), 
                      ("Дата запланированной поставки: 2023-04-30 (первая партия)", "30.04.2023"),
                      ("Дата запланированной поставки: первая партия: BZ: 2024-07-30; AD: 31 мая 2024 года.", "30.07.2024"),
                      ("Проект &lt; &lt; Старый ветер в Синьцзян-Тауне &gt; &gt; : 20 марта 2023 года (в соответствии с письменным уведомлением министерства)", "20.03.2023"),
                      ("Дата завершения проекта: 30 октября 2023 года", "30.10.2023")
                      ]
        ),
        Text(
            id="end_date",
            description="End date of work.",
            examples=[("Дата начала работ 15 мая 2023 года\nДата завершения проекта 31 июля 2023 года", "31.06.2023"), 
                      ]
        ),
    ],
    examples=[
        (
            """
            . ОБЩИЙ ОБЗОР ПРОЕКТОВ И ОГРАНИЧЕНИЙ
            Описание проекта: Закупка потребностей в исследованиях и разработках для проекта экспериментальной платформы электрической реле для защиты электрической реле, которая в основном предусматривала проведение научно-исследовательского проекта
            Программа включает общие требования к проектированию, материалам, изготовлению оборудования, проверке, испытанию, идентификации, оценке и контролю за документацией. Услуги или предметы
            Поставщики (далее "В"), помимо требований настоящего Технического регламента, должны отвечать существующим стандартам в стране и отрасли.
            Соответствующие положения.
            Содержание и сфера охвата тендера: в ходе конкурса было разработано 7 сканеров и комплектов для сканирования аптечек, шкафов и т.д., а также предложено
            Услуги по установке и отладке аппаратного обеспечения.
            Единица измерения количества серийных номеров
            Анализ потребностей в оборудовании 1
            2 аппаратных платформы для разработки 1
            1 шкаф для проверки 3 локомотивов и 1 шкаф для защиты маршрутов
            Оборудование для переключателя переключателя, защищенные карты
            14 единиц
            шкаф
            5 защитных карточек 1 шкаф
            1 шкаф для 6 карт защиты давления
            1 шкаф для 7-дюймовых магнитных карт
            8 универсальных реле 1 шкаф
            1 шкаф для 9 универсальных трансформаторов
            10 программных интерфейсов для разработки 1
            11 тестов для 1
            12 Установка и отладка.
            1
            13 документов, написанных сверху
            Дата запланированной поставки: 15 августа 2023 года - 31 декабря 2023 года.
            Общий срок службы: 138 календарных дней.
            """,
            [
                {"description": 'Закупка потребностей в исследованиях и разработках для проекта экспериментальной платформы электрической реле для защиты электрической реле, которая в основном предусматривала проведение научно-исследовательского проекта\nПрограмма включает общие требования к проектированию, материалам, изготовлению оборудования, проверке, испытанию, идентификации, оценке и контролю за документацией. Услуги или предметы\nПоставщики (далее "В"), помимо требований настоящего Технического регламента, должны отвечать существующим стандартам в стране и отрасли.', 
                 "requirements": 'в ходе конкурса было разработано 7 сканеров и комплектов для сканирования аптечек, шкафов и т.д., а также предложено\nУслуги по установке и отладке аппаратного обеспечения.',
                 "stages":"1",
                 "delivery_time": "138",
                 "start_date": "15.08.2023",
                 "end_date": "31.12.2023",
                 },
            ],
        ),
    ],
    many=True,
)

In [16]:
chain = create_extraction_chain(llm, schema, encoder_or_encoder_class='json')

In [18]:
result = []
for i_text in tqdm(df['part_2'][325:]):
    try:
        ans = chain.run(text=(i_text))
        result.append(ans['data'])
    except:
        result.append({'project_info': [ {"description": '', 
                                           "requirements": '',
                                           "stages":"",
                                           "delivery_time": "",
                                           "start_date": "",
                                           "end_date": "",}]}),


100%|██████████| 283/283 [1:18:31<00:00, 16.65s/it]


In [21]:
len(result)

608

In [25]:
temp_list = []
for i in tqdm(range(len(result))):
    try:
        if len(result[i]) > 0:
            temp_list.append(result[i]['project_info'][0])
        else:
            temp_list.append({'description': '',
                            'requirements': '',
                            'stages': '',
                            'delivery_time': '',
                            'start_date': '',
                            'end_date': ''})
    except:
        temp_list.append({'description': '',
                'requirements': '',
                'stages': '',
                'delivery_time': '',
                'start_date': '',
                'end_date': ''})

100%|██████████| 608/608 [00:00<00:00, 1882019.80it/s]


In [26]:
temp_df = pd.DataFrame(temp_list)

In [27]:
temp_df

,description,requirements,stages,delivery_time,start_date,end_date
0,"Проект <<Хиннин>>, расположенный в городе Цзян...",Этот тендер будет состоять из первого сегмента...,четыре сегмента: 1-й пункт: кузов; 2-й участок...,Предварительный 15 августа 2023 года; срок око...,15.08.2023,
1,Закупка оборудования для фотоаппаратов УНИТА в...,Участники торгов предоставили три комплекта ка...,2,до 10 июня 2023 года,,
2,Общий контрактный проект PC в 1 млн кВт-диапаз...,Упаковка 1: первый этап проекта генерального п...,четыре сегмента,Планируемый период работы составляет с 15 авгу...,15.08.2023,30.12.2023
3,Проект &lt; &lt; Старый ветер в Синьцзян-Тауне...,Содержание и сфера охвата данного тендера вклю...,347 календарных дней,347 календарных дней,28.02.2023,10.02.2024
4,С 2023 по 2024 гг. были собраны матрицы фотоэл...,Данный тендерный проект разделен на семь пункт...,7,NaN,2023,2024
...,...,...,...,...,...,...
603,На юго-западе юго-западного района Порт-Куньси...,В рамках этого тендера были закуплены ветряное...,"Вместимость проекта составляет 50 МВт, а общая...",20.05.2023,20.05.2023,20.05.2023
604,атомная электростанция порта Гуанси находится ...,"данный тендерный проект разделен на 1 пункт, к...",1,15.07.2023,15.07.2023,31.12.2025
605,новая энергетическая ветряная площадка &lt; &l...,4-й пункт: «Геронцзян Юнг». Поставки включают ...,4,20.08.2023,20.08.2023,NaN
606,&lt; &lt; Ядерная энергетика Китая и Китая &gt...,сейф для борьбы с пожарными клапанами ядерного...,NaN,1 сентября 2023 года,NaN,NaN


In [28]:
temp_df.to_csv('./data/part_2_for_merge.csv', index=False)